<a href="https://colab.research.google.com/github/Ninook-v/yandex__data_analyst_project-/blob/main/%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82_SQL_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B1%D0%B0%D0%B7%D1%8B_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%BA%D0%BD%D0%B8%D0%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Финальный проект SQL - Анализ базы данных книг


**Описание проекта:**

Компания купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.  

**Цель исследования:**

Проанализировать базу данных с помощью SQL-запросов

**Описание данных:**

Таблица `books`(Содержит данные о книгах):
* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.
  
Таблица `authors`(Содержит данные об авторах):
* author_id — идентификатор автора;
* author — имя автора.
  
Таблица `publishers`(Содержит данные об издательствах):
* publisher_id — идентификатор издательства;
* publisher — название издательства;

Таблица `ratings`(Содержит данные о пользовательских оценках книг):
* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.
  
Таблица `reviews`(Содержит данные о пользовательских обзорах на книги):
* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя пользователя, написавшего обзор;
* text — текст обзора.

**Задачи**
* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;
* Посчитайть среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.


In [ ]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [ ]:
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

## Обзор данных

In [ ]:
#напишем функцию для выведения первых 5 строк таклицы и общего количества записей
def get_table_info(table_name, engine):
    # ANSI код для жирного текста
    bold_start = '\033[1m'
    bold_end = '\033[0m'

    # Функция для выполнения запросов и вывода результатов

    # Получаем первые 5 записей из таблицы
    query = f'''
    SELECT *
    FROM {table_name}
    LIMIT 5;
    '''
    first_five_rows = pd.read_sql(query, con=engine)

    # Получаем число записей в таблице
    query = f'''
    SELECT COUNT(*)
    FROM {table_name};
    '''
    record_count = pd.read_sql(query, con=engine).iloc[0, 0]

    # Вывод результатов
    print(f"{bold_start}Первые 5 записей из таблицы {table_name}:{bold_end}")
    display(first_five_rows)
    print(f"{bold_start}Число записей в таблице {table_name}:{bold_end}{record_count}")

    return first_five_rows, record_count


### Таблица books

In [ ]:
get_table_info('books', engine);

Первые 5 записей из таблицы books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Число записей в таблице books:1000


### Таблица authors

In [ ]:
get_table_info('authors', engine);

Первые 5 записей из таблицы authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Число записей в таблице authors:636


### Таблица publishers

In [ ]:
get_table_info('publishers', engine);

Первые 5 записей из таблицы publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Число записей в таблице publishers:340


### Таблица ratings

In [ ]:
get_table_info('ratings', engine);

Первые 5 записей из таблицы ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Число записей в таблице ratings:6456


### Таблица reviews

In [ ]:
get_table_info('reviews', engine);

Первые 5 записей из таблицы reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Число записей в таблице reviews:2793


**Вывод:**
Ознакомились с таблицами в базе данных
* в базе имеется 1000 книг
* всего имеем данные о  636 авторах из таблицы `authors`
* в таблице`publishers`  есть данные о 340 издательствах
* в таблице об оценках пользователей `ratings` - 6456 строк
* в таблице  обзорах пользователей `reviews` - 2793 строки


## Задание 1. Посчитать, сколько книг вышло после 1 января 2000 года;

In [ ]:
query1 = '''
           SELECT COUNT(*) as count_book
           FROM books
           WHERE CAST(publication_date AS timestamp) > '2000-01-01'
        '''

pd.io.sql.read_sql(query1, con = engine)

,count_book
0,819


С января 2000 года вышло 819 книг

## Задание 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [ ]:
query2 = '''
           SELECT b.title,
                  COUNT(DISTINCT rw.review_id) AS count_reviews,
                  ROUND(AVG(rt.rating), 1) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS rw ON b.book_id = rw.book_id
           LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
           GROUP BY b.book_id
           ORDER BY avg_rating DESC,count_reviews DESC
        '''

pd.io.sql.read_sql(query2, con = engine)

,title,count_reviews,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.0
1,School's Out—Forever (Maximum Ride #2),3,5.0
2,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,Crucial Conversations: Tools for Talking When ...,2,5.0
4,Misty of Chincoteague (Misty #1),2,5.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.3
996,Drowning Ruth,3,2.0
997,His Excellency: George Washington,2,2.0
998,Junky,2,2.0


**Вывод:**
* Книга с самым большим количеством обзоров и наивысшим средним рейтингом- A Dirty Job (Grim Reaper #1) - 5,0
* Книга с самым низким средним рейтингом  -  Harvesting the Heart - 1,5

## Задание 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц
— так вы исключите из анализа брошюры

In [ ]:
query3 = '''
           SELECT p.publisher,
                  COUNT(b.book_id) AS books
           FROM publishers AS p
           JOIN books AS b ON p.publisher_id = b.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher_id
           ORDER BY books DESC
           LIMIT 1;
        '''

pd.io.sql.read_sql(query3, con = engine)

,publisher,books
0,Penguin Books,42


**Вывод:**\
Издательство `Penguin Books` выпустило наибольшее количество книг толще 50 страниц - **42 книги**

## Задание 4. Определить автора с самой высокой средней оценкой книг
— учитывайте  только книги с 50 и более оценками;

In [ ]:
query4 = '''
           SELECT a.author, ROUND(AVG(rt.rating),1) as avg_rating
           FROM authors AS a
           JOIN books AS b ON a.author_id = b.author_id
           JOIN ratings AS rt ON b.book_id = rt.book_id
           WHERE b.book_id IN (SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT(rating_id) >= 50
                               ORDER BY COUNT(rating_id) DESC)
           GROUP BY a.author
           ORDER BY avg_rating DESC
           LIMIT 1
        '''

pd.io.sql.read_sql(query4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.3


**Вывод:**\
`J.K. Rowling/Mary GrandPré	` - автор с самой высокой оценкой  книг, у которых 50 и более оценок - **рейтинг 4.3**
    
    

## Задание 5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [ ]:
query5 = '''
         SELECT AVG(res.cnt_review) AS avg_reviews
         FROM (SELECT rw.username, COUNT(rw.review_id) AS cnt_review
               FROM reviews AS rw
               WHERE rw.username IN (SELECT rt.username
                                    FROM ratings AS rt
                                    GROUP BY rt.username
                                    HAVING COUNT(rt.rating_id) > 48)
               GROUP BY rw.username) AS res
        '''

pd.io.sql.read_sql(query5, con = engine)

,avg_reviews
0,24.0


**Вывод:**\
Среднее количество обзоров от пользователей, которые поставили больше 48 оценок — 24.0   

## Общие выводы

Мы провели анализ базы данных, на основании которого можно сделать следующие выводы:

* Сервис для чтения хранит **1000** различных книг от 636 авторов
*  из них **819** книг выпустились с 2000 года
* рейтинг книг варьируется от **1.5 до 5.0**
   * Книга с самым большим количеством обзоров и наивысшим средним рейтингом- A Dirty Job (Grim Reaper #1) - 5,0
   * Книга с самым низким средним рейтингом - Harvesting the Heart - 1,5

* больше всего книг выпустило издательство Penguin Books - **42 книги**
- автором с самой высокой оценкой книги является J.K.Rowling/Mary GrandPré - **4.3**
- в среднем, самые активные пользователи в среднем **24 обзора**